In [18]:
from langchain_openai import OpenAI
from langchain_community.utilities import SQLDatabase

llm = OpenAI(model="gpt-4o-mini",temperature=0, verbose=True)
# res = llm.invoke("What is Langchain?")

db_user = "root"
db_password = ""
db_host = "localhost"
db_name = "cool_tshirts"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER(11) NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER(11) NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id)
)COLLATE utf8mb4_general_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER(11) NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER(11), 
	stock_quantity INTEGER(11) NOT NULL, 
	PRIMARY KEY (t_shirt_id)
)COLLATE utf8mb4_general_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Van Huesen	Red	M	31	41
2	Van Huesen	Black	L	42	74
3	Van Huesen	White	XS	40	43
*/


In [19]:
from langchain_experimental.sql import  SQLDatabaseChain

sql_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
result = sql_chain.run("Can I get van husen tshirt for my size which is 36 and it will be great if I get discount around 10 to 20 percemt? if discount is available then please provide me the discounted price ?")



> Entering new SQLDatabaseChain chain...
Can I get van husen tshirt for my size which is 36 and it will be great if I get discount around 10 to 20 percemt? if discount is available then please provide me the discounted price ?
SQLQuery:```sql
SELECT
  `t`.`t_shirt_id`,
  `t`.`brand`,
  `t`.`color`,
  `t`.`size`,
  `t`.`price`,
  `d`.`pct_discount`,
  `t`.`price` * (
    1 - `d`.`pct_discount` / 100
  ) AS discounted_price
FROM t_shirts AS `t`
JOIN discounts AS `d`
  ON `t`.`t_shirt_id` = `d`.`t_shirt_id`
WHERE
  `t`.`brand` = 'Van Huesen' AND `d`.`pct_discount` BETWEEN 10 AND 20
LIMIT 5;
```

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near '```sql\nSELECT\n  `t`.`t_shirt_id`,\n  `t`.`brand`,\n  `t`.`color`,\n  `t`.`size`,...' at line 1")
[SQL: ```sql
SELECT
  `t`.`t_shirt_id`,
  `t`.`brand`,
  `t`.`color`,
  `t`.`size`,
  `t`.`price`,
  `d`.`pct_discount`,
  `t`.`price` * (
    1 - `d`.`pct_discount` / 100
  ) AS discounted_price
FROM t_shirts AS `t`
JOIN discounts AS `d`
  ON `t`.`t_shirt_id` = `d`.`t_shirt_id`
WHERE
  `t`.`brand` = 'Van Huesen' AND `d`.`pct_discount` BETWEEN 10 AND 20
LIMIT 5;
```]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [20]:
result1 = sql_chain.run("How much is the price of the inventory for all small size tshirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size tshirts?
SQLQuery:```sql
SELECT SUM(`price` * `stock_quantity`) AS inventory_value
FROM `t_shirts`
WHERE `size` = 'S';
```

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near '```sql\nSELECT SUM(`price` * `stock_quantity`) AS inventory_value\nFROM `t_shir...' at line 1")
[SQL: ```sql
SELECT SUM(`price` * `stock_quantity`) AS inventory_value
FROM `t_shirts`
WHERE `size` = 'S';
```]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [7]:
result2 = sql_chain.run("SELECT SUM(('price'*'stock_quantity') * (1 - 'pct_discount' / 100)) AS 'revenue' FROM 't_shirts' JOIN `discounts` ON `t_shirts`.`t_shirt_id` = `discounts`.`t_shirt_id` WHERE `brand` = 'Nike' AND `stock_quantity` > 0;")



> Entering new SQLDatabaseChain chain...
SELECT SUM(('price'*'stock_quantity') * (1 - 'pct_discount' / 100)) AS 'revenue' FROM 't_shirts' JOIN `discounts` ON `t_shirts`.`t_shirt_id` = `discounts`.`t_shirt_id` WHERE `brand` = 'Nike' AND `stock_quantity` > 0;
SQLQuery:

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-xaCfc47s7P0fPQ1iOYYK2RFv on tokens per min (TPM): Limit 100000, Used 99709, Requested 570. Please try again in 2h0m31.68s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [1]:
# result3 = sql_chain.run("How many white colors levi's t-shirts we have available?")
result3 = sql_chain.run("SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `color` = 'White' AND `brand` = 'Levi'")


NameError: name 'sql_chain' is not defined